# 10 Date transformation


In [1]:
import pandas as pd

In [3]:
from datetime import date

pdf = pd.DataFrame({'date':
['01-01-2017',
'04-12-2008',
'23-06-1988',
'25-08-1999',
'20-02-1993',
]})

#Transform string to date
pdf['date'] = pd.to_datetime(pdf.date, format="%d-%m-%Y")

In [4]:
pdf.head()

,date
0,2017-01-01
1,2008-12-04
2,1988-06-23
3,1999-08-25
4,1993-02-20


# 10.1 Explode date

Extracting the parts of the date into different columns: Year, month, day

In [5]:
res=pdf.copy()
#Extracting Year
res['year'] = res['date'].dt.year

res.head()


,date,year
0,2017-01-01,2017
1,2008-12-04,2008
2,1988-06-23,1988
3,1999-08-25,1999
4,1993-02-20,1993


In [6]:
#Extracting Month
res['month'] = res['date'].dt.month

res.head()

,date,year,month
0,2017-01-01,2017,1
1,2008-12-04,2008,12
2,1988-06-23,1988,6
3,1999-08-25,1999,8
4,1993-02-20,1993,2


In [7]:
#Extracting day
res['day'] = res['date'].dt.day

res.head()

,date,year,month,day
0,2017-01-01,2017,1,1
1,2008-12-04,2008,12,4
2,1988-06-23,1988,6,23
3,1999-08-25,1999,8,25
4,1993-02-20,1993,2,20


## 10.2 Calculate the time period

Extracting the time period between the current date and columns in terms of years, months, days, etc.

In [8]:
period=pdf.copy()

#Extracting passed years since the date
period['passed_years'] = date.today().year - period['date'].dt.year
period.head()

,date,passed_years
0,2017-01-01,5
1,2008-12-04,14
2,1988-06-23,34
3,1999-08-25,23
4,1993-02-20,29


In [9]:
#Extracting passed months since the date
period['passed_months'] = (date.today().year - period['date'].dt.year) * 12 + date.today().month - period['date'].dt.month

period.head()

,date,passed_years,passed_months
0,2017-01-01,5,66
1,2008-12-04,14,163
2,1988-06-23,34,409
3,1999-08-25,23,275
4,1993-02-20,29,353


## 10.3 Get special day marks

Extracting some specific features from the date: Name of the weekday, Weekend or not, holiday or not, etc.

In [10]:
special_day=pdf.copy()


#Extracting the weekday name of the date
special_day['day_name'] = special_day['date'].dt.day_name()

special_day.head()

,date,day_name
0,2017-01-01,Sunday
1,2008-12-04,Thursday
2,1988-06-23,Thursday
3,1999-08-25,Wednesday
4,1993-02-20,Saturday
